딥 러닝 (심층 학습)
===================

과제 2
------

이전 과제 `1_notmnist_korean.ipynb`에서는 [notMNIST 데이터셋](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html)을 사용해서 정형화된 학습용, 개발용, 테스트용 데이터셋을 피클 형식으로 만들었습니다.

이번 과제의 목적은 텐서플로우를 사용해서 모델을 점점 더 심도있고 정확하게 학습시키는 것입니다.

In [1]:
# 모두 앞으로 사용할 모듈들입니다.
# 더 진행하기 전에, 모든 모듈을 불러올 수 있는지 확인하세요.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

print('역자 주: 이 메시지가 보이면 잘 된 것입니다.')

역자 주: 이 메시지가 보이면 잘 된 것입니다.


먼저 `1_notmnist_korean.ipynb`에서 만든 데이터를 다시 불러옵시다.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # 쓰레기 수집기에게 메모리를 해제하라고 알려줍니다.
  print('학습용 데이터', train_dataset.shape, train_labels.shape)
  print('검증용 데이터', valid_dataset.shape, valid_labels.shape)
  print('테스트용 데이터', test_dataset.shape, test_labels.shape)

학습용 데이터 (200000, 28, 28) (200000,)
검증용 데이터 (10000, 28, 28) (10000,)
테스트용 데이터 (10000, 28, 28) (10000,)


데이터셋을 앞으로 우리가 학습시킬 모델에 더욱 적합한 형태로 변형합시다.
- 데이터는 평면 행렬로 변환합니다. (역자 주: 2차원 행렬을 말하는 것 같습니다.)
- 라벨은 원 핫 인코딩으로 변환합니다.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # 0은 [1.0, 0.0, 0.0 ...]으로, 1은 [0.0, 1.0, 0.0 ...]으로 변환합니다.
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('학습용 데이터', train_dataset.shape, train_labels.shape)
print('검증용 데이터', valid_dataset.shape, valid_labels.shape)
print('테스트용 데이터', test_dataset.shape, test_labels.shape)

학습용 데이터 (200000, 784) (200000, 10)
검증용 데이터 (10000, 784) (10000, 10)
테스트용 데이터 (10000, 784) (10000, 10)


먼저 간단한 경사 하강법을 사용해서 다항 로지스틱 회귀를 학습시켜 보도록 하겠습니다.

텐서플로우의 방식은 다음과 같습니다.
* 먼저 여러분이 실행하고 싶은 연산을 기술해야 합니다. 입력은 어떻게 생겼는지, 변수는 어떻게 생겼는지, 어떤 작업을 해야 하는지 등에 대해서입니다. 이 내용이 계산 그래프 위에 노드로 생성됩니다. 이 모든 설명은 아래 블록에 포함되어 있습니다.

      with graph.as_default():
          ...

* 그 후 이 그래프에 있는 작업을 `session.run()`을 사용해서 원하는 만큼 계속 반복하는데, 세션에 그래프에서 가져온 출력값을 제공해야 합니다. 이 런타임 작업은 아래 블록에 모두 포함되어 있습니다.

      with tf.Session(graph=graph) as session:
          ...

텐서플로우에서 모든 데이터를 불러온 뒤 우리의 학습을 위한 계산 그래프를 만들어 봅시다.

In [6]:
# 경사 하강법 학습에서는 이정도의 데이터 양도 많습니다.
# 빨리 진행되도록 학습 데이터를 조금만 뽑습니다.
# (역자 주: 만약 다음 블록에서 Dst tensor is not initialized 에러가 나면
# train_subset 크기를 줄이세요.)
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # 입력 데이터
  # 학습용, 검증용, 테스트용 데이터를 불러온 뒤 그래프에 연동된 변수에 할당합니다.  
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # 변수
  # 우리가 학습시킬 대상에 대한 매개변수입니다.
  # 가중치 행렬은 (절삭된) 정규분포를 따르는 무작위 값으로 초기화됩니다.
  # 편향값은 0으로 초기화됩니다.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # 학습 연산
  # 입력값에 가중치 행렬을 곱하고 편향값을 더합니다.
  # 소프트맥스와 크로스 엔트로피를 계산합니다.
  # (텐서플로우에서는 한 번의 작업으로 가능합니다. 흔하게 사용되기 때문이고, 최적화될 수 있습니다.)
  # 크로스 엔트로피의 평균값을 모든 학습용 예제에 대해 구합니다. 이것이 우리의 손실값이 됩니다.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # 최적화
  # 이 손실값의 최소값을 경사 하강법을 사용해서 찾아냅니다.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # 학습용, 검증용, 테스트용 데이터에 대한 예측값
  # 이 단계는 학습의 일부가 아닙니다. 학습에 대한 정확도를 알기 위한 단계입니다.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

이 계산을 반복해서 실행해봅시다.

In [7]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # 이 부분은 한 번만 수행되는 부분으로, 매개변수들이 우리가 그래프에서 정의한 대로 초기화되도록 합니다.
  # 행렬에는 무작위 가중치, 편향값은 0입니다.
  tf.global_variables_initializer().run()
  print('초기화 되었습니다.')
  for step in range(num_steps):
    # 계산을 합니다. 최적화를 수행하기 위해 .run()을 사용합니다.
    # 손실값과 학습 데이터에 대한 예측값을 numpy 배열로 받습니다.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('%d 단계에서의 손실: %f' % (step, l))
      print('학습 정확도: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # valid_prediction에 대해 .eval()을 실행하는 것은 run()을 부르는 것과 본질적으로는 같습니다.
      # 하지만 딱 하나의 numpy 배열만 얻게 됩니다.
      # 그래프가 의존성을 갖는 모든 값이 다시 다 계산된다는 것에 주의하세요.
      print('검증 정확도: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('테스트 정확도: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

초기화 되었습니다.
0 단계에서의 손실: 17.918882
학습 정확도: 12.9%
검증 정확도: 15.3%
100 단계에서의 손실: 2.294772
학습 정확도: 71.8%
검증 정확도: 71.4%
200 단계에서의 손실: 1.857128
학습 정확도: 75.0%
검증 정확도: 73.5%
300 단계에서의 손실: 1.619935
학습 정확도: 76.0%
검증 정확도: 74.4%
400 단계에서의 손실: 1.458887
학습 정확도: 76.7%
검증 정확도: 74.8%
500 단계에서의 손실: 1.338105
학습 정확도: 77.5%
검증 정확도: 75.2%
600 단계에서의 손실: 1.242423
학습 정확도: 78.0%
검증 정확도: 75.2%
700 단계에서의 손실: 1.163966
학습 정확도: 78.6%
검증 정확도: 75.4%
800 단계에서의 손실: 1.098125
학습 정확도: 79.2%
검증 정확도: 75.6%
테스트 정확도: 82.6%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [0]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [0]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 16.8091
Minibatch accuracy: 12.5%
Validation accuracy: 14.0%
Minibatch loss at step 500 : 1.75256
Minibatch accuracy: 77.3%
Validation accuracy: 75.0%
Minibatch loss at step 1000 : 1.32283
Minibatch accuracy: 77.3%
Validation accuracy: 76.6%
Minibatch loss at step 1500 : 0.944533
Minibatch accuracy: 83.6%
Validation accuracy: 76.5%
Minibatch loss at step 2000 : 1.03795
Minibatch accuracy: 78.9%
Validation accuracy: 77.8%
Minibatch loss at step 2500 : 1.10219
Minibatch accuracy: 80.5%
Validation accuracy: 78.0%
Minibatch loss at step 3000 : 0.758874
Minibatch accuracy: 82.8%
Validation accuracy: 78.8%
Test accuracy: 86.1%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---